In [499]:
import string
from unidecode import unidecode
import time;
import datetime
import numpy as np
import pandas as pd
import os
import re

import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from transformers import AdamW, BertModel, AutoConfig, AutoTokenizer
from torch.optim import Adam
import torch.nn as nn
from torchcrf import CRF
from transformers import AutoTokenizer, get_linear_schedule_with_warmup

from sklearn.model_selection import train_test_split

import zemberek
from zemberek.morphology import TurkishMorphology
from zemberek.morphology.analysis.rule_based_analyzer import RuleBasedAnalyzer
from zemberek.normalization import TurkishSpellChecker
from zemberek.morphology.morphotactics import TurkishMorphotactics, InformalTurkishMorphotactics
#import RootLexicon
from zemberek.morphology.lexicon import RootLexicon

In [500]:
# !pip install pytorch-crf
# !pip install unidecode
# !pip install --upgrade transformers
# !pip install --upgrade torch
# !pip install zemberek-python

In [501]:
if torch.cuda.is_available():  
    device = torch.device("cuda")
    print('We will use the GPU:', torch.cuda.get_device_name(0))
    
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [502]:
# nlp_train=pd.read_csv("/kaggle/input/nlp-project-train/train.csv", index_col=[0])
# nlp_test=pd.read_csv("/kaggle/input/nlp-project-train/test.csv",index_col=[0],encoding="windows-1252") 

nlp_train=pd.read_csv("train.csv", index_col=[0])
nlp_test=pd.read_csv("test.csv",index_col=[0],encoding="windows-1252") 

nlp_train = nlp_train.iloc[:2]

In [503]:
lower_chars_dict = {
    "A": "a",
    "B": "b",
    "C": "c",
    "Ç": "ç",
    "D": "d",
    "E": "e",
    "F": "f",
    "G": "g",
    "Ğ": "ğ",
    "H": "h",
    "I": "ı",
    "İ": "i",
    "J": "j",
    "K": "k",
    "L": "l",
    "M": "m",
    "N": "n",
    "O": "o",
    "Ö": "ö",
    "P": "p",
    "R": "r",
    "S": "s",
    "Ş": "ş",
    "T": "t",
    "U": "u",
    "Ü": "ü",
    "V": "v",
    "Y": "y",
    "Z": "z",
    "Q": "q",
    "W": "w",
    "X": "x"    
    }
    
words_dict = {
    "ı": "i",
    "ğ": "g",
    "ü": "u",
    "ş": "s",
    "ö": "o",
    "ç": "c",
    "İ": "I",
    "Ğ": "G",
    "Ü": "U",
    "Ş": "S",
    "Ö": "O",
    "Ç": "C"
    }

characters_dictionary = {
    "a":1,
    "b":2,
    "c":3,
    "ç":4,
    "d":5,
    "e":6,
    "f":7,
    "g":8,
    "ğ":9,
    "h":10,
    "ı":11,
    "i":12,
    "j":13,
    "k":14,
    "l":15,
    "m":16,
    "n":17,
    "o":18,
    "ö":19,
    "p":20,
    "r":21,
    "s":22,
    "ş":23,
    "t":24,
    "u":25,
    "ü":26,
    "v":27,
    "y":28,
    "z":29,
    "q": 30,
    "x":31,
    "w":32,
    " ":33,
    }
    


## Prepare train data

In [517]:
def convert_to_ascii(sentence):
    # convert Turkish characters to English characters
    for key, value in words_dict.items():
        sentence = sentence.replace(key, value)
    return sentence

def remove_puncutations_and_numbers(text):
    # Iterate over the string and remove char if it is not a character
    characters = "abcçdefgğhıijklmnoöprsştuüvyzABCÇDEFGĞHIİJKLMNOÖPRSŞTUÜVYZxXwWqQ "
    for char in text:
        if char not in characters:
            text = text.replace(char, "")
    # Remove multiple spaces
    text = re.sub(' +', ' ', text)
    # Lowercase the text
    # text = text.lower()
    return text

def split_sentences(sentences, max_length=100):
    punctuations = {'.', ',', ';', ':'}
    results = []

    for sentence in sentences:
        while len(sentence) > max_length:
            # find last punctuation before max_length
            split_pos = -1
            for p in punctuations:
                pos = sentence.rfind(p, 0, max_length + 1)
                if pos > split_pos:
                    split_pos = pos
            
            # If no punctuation found, split at the last space before max_length
            if split_pos == -1:
                split_pos = sentence.rfind(' ', 0, max_length + 1)
            
            # If no space found, just split at max_length
            if split_pos == -1:
                split_pos = max_length
            
            # Append the split segment to results
            results.append(sentence[:split_pos + 1].strip())
            # Move the rest of the sentence forward
            sentence = sentence[split_pos + 1:].strip()
        
        # Append the remainder of the sentence if it's not empty
        if sentence:
            results.append(sentence)
    
    return results

def padding(text, filling_char, max_length):
    if type(text) is str:
        text = text + filling_char * (max_length - len(text))
    elif type(text) is list:
        text = text + [filling_char] * (max_length - len(text))
    return text

def map_diacritics(text):
    for char_index in range(len(text)):
        # print(text[char_index])
        if text[char_index] in "ıöüğşç":
            text[char_index] = 2
        elif text[char_index] in "iougsc":
            text[char_index] = 1
        elif text[char_index] == " ":
            text[char_index] = 0
        else:
            text[char_index] = 3
        
    return text

def prepare_train_dataset(sentences):
    print(sentences)
    processed_sentences = []
    # Iterate over the sentences
    for i, sentence in enumerate(sentences):
        # Remove punctuations and numbers
        sentence = remove_puncutations_and_numbers(sentence)
        # Split the sentence into smaller sentences
        splitted_sentences = split_sentences([sentence])
        # print(splitted_sentences)
        # break
        new_list = []
        # Iterate over the splitted sentences
        for s in splitted_sentences:
            # Lowerize characters
            for key, value in lower_chars_dict.items():
                s = s.replace(key, value)
            new_list.append(s)
        processed_sentences.extend(new_list)
    print(processed_sentences)
    
    diactrize_labels = processed_sentences.copy()
    # Iterate over the sentences to diacritize them
    for i, sentence in enumerate(processed_sentences):
        diactrize_labels[i] = map_diacritics(list(sentence))
        # Add padding to the diacritized sentence
        diactrize_labels[i] = padding(diactrize_labels[i], 0, 100)
    # Asci sentences 
    asci_sentences = processed_sentences.copy()
    # Iterate over the sentences to convert them to asci and map them to numbers
    for i, sentence in enumerate(processed_sentences):
        asci_sentences[i] = convert_to_ascii(sentence)
        
    numeric_sentences = []
    for i, sentence in enumerate(asci_sentences):
        numeric_sentence = []
        for char in sentence:
            numeric_sentence.append(characters_dictionary[char])
        # Add padding to the numeric sentence
        numeric_sentence = padding(numeric_sentence, 0, 100)
        numeric_sentences.append(numeric_sentence)
    return processed_sentences, diactrize_labels, asci_sentences, numeric_sentences

In [518]:
turkish_sentences = [
    "uzun zamandır görüşemedik, nasılsın?",
    "bu nasıl bir soru?",
    "Bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak. Devam ediyoruz çünkü bu cümle hala bitmedi. Bitmedi bitmeyecek. Çünkü bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak. Devam ediyoruz çünkü bu cümle hala bitmedi. Bitmedi bitmeyecek. Devam et",
]

In [519]:
# Raw sentences
raw_sentences = nlp_train["Sentence"].values

processed_turkish_sentences,labels,asci_sentences,numeric_sentences = prepare_train_dataset(turkish_sentences.copy())

['uzun zamandır görüşemedik, nasılsın?', 'bu nasıl bir soru?', 'Bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak. Devam ediyoruz çünkü bu cümle hala bitmedi. Bitmedi bitmeyecek. Çünkü bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak. Devam ediyoruz çünkü bu cümle hala bitmedi. Bitmedi bitmeyecek. Devam et']
['uzun zamandır görüşemedik nasılsın', 'bu nasıl bir soru', 'bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak devam ediyoruz çünkü bu', 'cümle hala bitmedi bitmedi bitmeyecek çünkü bu cümle çok uzun bir cümle olacak ve bu cümlede çok', 'fazla kelime olacak devam ediyoruz çünkü bu cümle hala bitmedi bitmedi bitmeyecek devam et']


In [392]:
class DiacritizationBiLSTMCRF(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, num_labels):
        super(DiacritizationBiLSTMCRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.lstm = nn.LSTM(embed_dim, hidden_dim // 2, num_layers=1, bidirectional=True, batch_first=True)
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(hidden_dim, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, sentences, labels=None):
        # Embedding layer
        x = self.embedding(sentences)
        
        # BiLSTM layer
        x, _ = self.lstm(x)
        x = self.dropout(x)
        
        # Fully connected layer
        emissions = self.fc(x)
        # CRF layer
        if labels is not None:
            # If labels are provided, calculate the loss
            loss = -self.crf(emissions, labels)
            return loss
        else:
            # Otherwise, return the best path
            prediction = self.crf.decode(emissions)
            # get probabilites of the best path
            return prediction

In [393]:
# Example of initializing the model
vocab_size = len(characters_dictionary) + 1  # Number of unique characters in your character dictionary + 1 for padding
embed_dim = 128
hidden_dim = 256
num_labels = 4  # Diacritic or not
model = DiacritizationBiLSTMCRF(vocab_size, embed_dim, hidden_dim, num_labels)
model.to(device)

DiacritizationBiLSTMCRF(
  (embedding): Embedding(34, 128)
  (lstm): LSTM(128, 128, batch_first=True, bidirectional=True)
  (dropout): Dropout(p=0.1, inplace=False)
  (fc): Linear(in_features=256, out_features=4, bias=True)
  (crf): CRF(num_tags=4)
)

In [394]:
numeric_sentences_tensor = torch.tensor(numeric_sentences, dtype=torch.long)
labels_tensor = torch.tensor(labels, dtype=torch.long)

# Splitting three variables: numeric_sentences_tensor, labels_tensor, and another_tensor
X_train, X_val, y_train, y_val, sentences_train, sentences_val = train_test_split(
    numeric_sentences_tensor, labels_tensor, asci_sentences, test_size=0.2, random_state=42)

# Create TensorDataset and DataLoader for training and validation
train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Define the loss function and the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [395]:
# Training the model
def train_model(model, data_loader, optimizer, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for sentences, label_seqs in data_loader:
            sentences, label_seqs = sentences.to(device), label_seqs.to(device)
            
            # Forward pass: Compute predicted y by passing x to the model
            loss = model(sentences, labels=label_seqs)
            
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(data_loader)}")

In [396]:
# Start training
#train_model(model, train_loader, optimizer, num_epochs=10)

In [558]:
def predict_test_sentence(sentence):
    print(sentence)
    # sentence = [sentence]
    # Remove punctuations and numbers
    sentence = remove_puncutations_and_numbers(sentence)
    print(sentence)
    
    # Lowerize characters
    for key, value in lower_chars_dict.items():
        sentence = sentence.replace(key, value)
    print(sentence)
    
    # Convert to asci
    sentence = convert_to_ascii(sentence)
    print(sentence)
    
    sentences_array = []
    # Split sentence into smaller sentences by using split_sentences function
    sentences_array.extend(split_sentences([sentence]))
    print(sentences_array)
    # Map to numbers
    numeric_sentences = []
    for sentence in sentences_array:
        numeric_sentence = []
        print(len(sentence))
        for char in sentence:
            numeric_sentence.append(characters_dictionary[char])
        numeric_sentences.append(numeric_sentence)
        
    
    # Add padding
    # print(numeric_sentence)
    return numeric_sentences

In [559]:
uzun_cumle = "Bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak. Devam ediyoruz çünkü bu cümle hala bitmedi. Bitmedi bitmeyecek. Çünkü bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak. Devam ediyoruz çünkü bu cümle hala bitmedi. Bitmedi bitmeyecek. Devam et. Devam et"
new_sentence = predict_test_sentence(uzun_cumle)
print(new_sentence)

Bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak. Devam ediyoruz çünkü bu cümle hala bitmedi. Bitmedi bitmeyecek. Çünkü bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak. Devam ediyoruz çünkü bu cümle hala bitmedi. Bitmedi bitmeyecek. Devam et. Devam et
Bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak Devam ediyoruz çünkü bu cümle hala bitmedi Bitmedi bitmeyecek Çünkü bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak Devam ediyoruz çünkü bu cümle hala bitmedi Bitmedi bitmeyecek Devam et Devam et
bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak devam ediyoruz çünkü bu cümle hala bitmedi bitmedi bitmeyecek çünkü bu cümle çok uzun bir cümle olacak ve bu cümlede çok fazla kelime olacak devam ediyoruz çünkü bu cümle hala bitmedi bitmedi bitmeyecek devam et devam et
bu cumle cok uzun bir cumle olacak ve bu cumlede cok fazla kelime olacak devam ediyoruz cunku bu cumle hala bit

In [397]:
def preprocess_sentence(sentence, characters_dictionary, max_length=200):
    # Remove punctuation and numbers
    sentence = ''.join([i for i in sentence if not i.isdigit() and i not in string.punctuation])
    sentence = sentence.lower().strip()
    
    # Convert to numeric form using the character dictionary
    numeric_sentence = [characters_dictionary.get(char, 0) for char in sentence]  # default to 0 if char not found
    
    # Padding
    if len(numeric_sentence) > max_length:
        numeric_sentence = numeric_sentence[:max_length]
    else:
        numeric_sentence += [0] * (max_length - len(numeric_sentence))
    
    return torch.tensor([numeric_sentence], dtype=torch.long).to(device)

def predict(model, sentence, characters_dictionary):
    model.eval()
    with torch.no_grad():
        numeric_sentence = preprocess_sentence(sentence, characters_dictionary)
        prediction = model(numeric_sentence)
        return prediction[0]  # since we have a batch size of 1

In [398]:
# Example sentence
test_sentence = "imrali Mit gorusmesi ihtiyac duyuldukca 12452 oluyor"
model = DiacritizationBiLSTMCRF(vocab_size, embed_dim, hidden_dim, num_labels)
model.to(device)
# Load the saved weights
model.load_state_dict(torch.load('/Users/mustafa/Desktop/Courses/Natural Language Processing/Project/Code/YZV405_2324_150200326_150210339/model_epoch_last.pth', map_location=torch.device('cpu')))
# Predict using the model
predicted_labels = predict(model, test_sentence, characters_dictionary)
print(predicted_labels)

# Define labels to diacritics mapping
def labels_to_diacritics(text_sentence, labels):
    print(text_sentence)
    diacritics_map_dict = {"i": "ı", "o": "ö", "u": "ü", "g": "ğ", "s": "ş", "c": "ç", "I": "İ"}
    output_sentence = ""
    for i in range(len(text_sentence)):
        if labels[i] == 2:
            if text_sentence[i] in diacritics_map_dict:
                output_sentence += diacritics_map_dict[text_sentence[i]]
            else:
                output_sentence += text_sentence[i]
        else:
            output_sentence += text_sentence[i]
    return output_sentence

# Show the results
output_sentence = labels_to_diacritics(test_sentence, predicted_labels)
print("Original:", test_sentence)
print("Processed:", output_sentence)

[1, 3, 3, 3, 3, 2, 0, 3, 1, 3, 0, 1, 2, 3, 2, 2, 3, 3, 1, 1, 0, 1, 3, 3, 1, 3, 3, 2, 0, 3, 1, 3, 1, 3, 3, 1, 3, 2, 3, 0, 0, 1, 3, 1, 3, 1, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
imrali Mit gorusmesi ihtiyac duyuldukca 12452 oluyor
Original: imrali Mit gorusmesi ihtiyac duyuldukca 12452 oluyor
Processed: imralı Mit görüşmesi ihtiyaç duyuldukça 12452 oluyor


In [400]:
def acc_overall(test_result, testgold):
  correct = 0
  total = 0
  print(test_result)
  print(testgold)
  # count number of correctly diacritized words
  for i in range(len(testgold)):
    for m in range(len(testgold[i].split())):
      if test_result[i].split()[m] == testgold[i].split()[m]:
        correct += 1
      total +=1

  return correct / total

In [401]:
diacritic_versions = {"i":"ı", "ı":"i", "o":"ö", "ö":"o", "u":"ü", "ü":"u", "g":"ğ", "ğ":"g", "s":"ş", "ş":"s", "c":"ç", "ç":"c"}

In [402]:
def check_morphology(sentences):
    morphology = TurkishMorphology.create_with_defaults()
    morphology.ignoreDiacriticsInAnalysis = False
    spellChecker = TurkishSpellChecker(morphology)

    corrected_sentences = sentences.copy()

    # Iterate through each sentence
    for i in range(len(corrected_sentences)):
        # Split the sentence into words and iterate through each word
        words = corrected_sentences[i].split()
        for m in range(len(words)):
            analysis = morphology.analyze(words[m])

            # Check if the word has no analysis results
            if len(analysis.analysis_results) == 0:
                print("No analysis results for:", words[m])
                
                # Check if there are suggestions from the spell checker
                suggestions = spellChecker.suggest_for_word(words[m])
                if len(suggestions) != 0:
                    for suggested_word in suggestions:
                        print("Suggested word:", suggested_word)                  
                        # if the suggested word has the same length as the original word, and just "ıioöuügğsşcç" characters are different, replace the word
                        if len(suggested_word) == len(words[m]):
                            for char1, char2 in zip(suggested_word, words[m]):
                                if char1 in diacritic_versions and char2 == diacritic_versions[char1]:
                                    continue
                                if char1 in diacritic_versions and char2 == char1:
                                    continue
                                if char1 not in diacritic_versions and char1 == char2:
                                    continue
                                elif char1 in diacritic_versions and char2 != diacritic_versions[char1] and char2 != char1:
                                    break
                                elif char1 not in diacritic_versions and char1 != char2:
                                    break
                            else:
                                print("Suggested word is approved:", suggested_word)
                                words[m] = suggested_word
                                break

        # Join the modified words back into a sentence
        corrected_sentences[i] = ' '.join(words)

    return corrected_sentences

In [403]:
def test_model(model, val_loader, characters_dictionary):
    model.eval()
    predicted_results = []
    true_results = []

    for (sentences, label_seqs), real_sentence in zip(val_loader, sentences_val):
        sentence = sentences.squeeze(0)  # Remove the batch dimension
        sentence = sentence.tolist()  # Convert tensor to list of sentences
        print(sentence)
        # Preprocess each sentence and obtain predicted labels
        model.eval()
        with torch.no_grad():
            prediction = model(sentences)
        predicted_sentence = labels_to_diacritics(real_sentence, prediction[0])
        
        # Append predicted and true results
        predicted_results.append(predicted_sentence)
        true_results.append(labels_to_diacritics(real_sentence, label_seqs.squeeze(0).tolist()))

    last_version = check_morphology(predicted_results)
    print("Last version:", last_version)
    # Calculate accuracy
    accuracy = acc_overall(last_version, true_results)
    return accuracy

In [404]:
# Test the model on the validation dataset
accuracy = test_model(model, val_loader, characters_dictionary)
print("Validation Accuracy:", accuracy)

[2, 1, 22, 1, 21, 12, 15, 12, 33, 2, 12, 21, 33, 22, 24, 1, 17, 5, 1, 21, 24, 33, 18, 15, 25, 22, 24, 25, 21, 16, 25, 22, 24, 25, 21, 33, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
basarili bir standart olusturmustur 
basarili bir standart olusturmustur 


2024-05-06 21:37:20,989 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 4.086855888366699

Last version: ['başarılı bir standart oluşturmuştur']
['başarılı bir standart oluşturmuştur']
['başarılı bir standart oluşturmuştur ']
Validation Accuracy: 1.0


In [405]:
def predict_test(model, test_sentences, characters_dictionary, max_length=100):
    model.eval()
    predicted_results = []
    

    for sentence in test_sentences:
        # Store the original sentence
        original_sentence = sentence
        
        sentence = ["Çok uzun bir cümle olacak ve bu cümleyi parçalara ayırmamız gerekecek. Çünkü modelimiz çok uzun cümleleri işleyemiyor. Bu yüzden cümleyi parçalara ayırıp işleyeceğiz. Bu işlem sonucunda cümledeki kelimelerin üzerindeki ağırlıkların doğru tahmin edilmesini bekliyoruz. Modelimizin bu işlemi başarılı bir şekilde yapacağına inanıyoruz. Umarım sonuçlar bizi yanıltmaz. Çünkü bu sonuçlar bizim için çok önemli. "]
        # Split long sentences into manageable parts by using split_sentences function
        print(len(sentence))
        parts = split_sentences(sentence, max_length)
        # 
        print(parts)
        break
        
        predicted_sentence = ""

        for part in parts:
            # Preprocess and predict for each part
            numeric_sentence = preprocess_sentence(part, characters_dictionary, max_length)
            numeric_part = torch.tensor([numeric_sentence], dtype=torch.long).to(device)
            prediction = predict(model, numeric_part, characters_dictionary)
            # Apply diacritics to each part and combine
            predicted_part = labels_to_diacritics(part, prediction)
            predicted_sentence += predicted_part

        predicted_results.append(predicted_sentence)

    return predicted_results

# Function to preprocess and convert a sentence to its numeric form
def preprocess_sentence(sentence, characters_dictionary, max_length):
    numeric_sentence = [characters_dictionary.get(char, 0) for char in sentence.lower()]
    # Padding
    if len(numeric_sentence) > max_length:
        numeric_sentence = numeric_sentence[:max_length]
    else:
        numeric_sentence += [0] * (max_length - len(numeric_sentence))
    return numeric_sentence

# Helper function to handle predictions for a numeric sentence
def predict(model, numeric_sentence, characters_dictionary):
    model.eval()
    with torch.no_grad():
        prediction = model(numeric_sentence)
    return prediction[0]  # Assuming batch size of 1 for simplicity


# Make predictions on the test set
test_sentences = nlp_test["Sentence"].values
predicted_results = predict_test(model, test_sentences, characters_dictionary)
print(predicted_results)


1
['Çok uzun bir cümle olacak ve bu cümleyi parçalara ayırmamız gerekecek.'
 'Çünkü modelimiz çok uzun cümleleri işleyemiyor. Bu yüzden cümleyi parçalara ayırıp işleyeceğiz.'
 'Bu işlem sonucunda cümledeki kelimelerin üzerindeki ağırlıkların doğru tahmin edilmesini bekliyoruz.'
 'Modelimizin bu işlemi başarılı bir şekilde yapacağına inanıyoruz. Umarım sonuçlar bizi yanıltmaz.'
 'Çünkü bu sonuçlar bizim için çok önemli.']
[]


In [406]:
# Make predictions on the test set
test_sentences = nlp_test["Sentence"].values
predicted_results = predict_test(model, test_sentences, characters_dictionary)
print(predicted_results)

1
['Çok uzun bir cümle olacak ve bu cümleyi parçalara ayırmamız gerekecek.'
 'Çünkü modelimiz çok uzun cümleleri işleyemiyor. Bu yüzden cümleyi parçalara ayırıp işleyeceğiz.'
 'Bu işlem sonucunda cümledeki kelimelerin üzerindeki ağırlıkların doğru tahmin edilmesini bekliyoruz.'
 'Modelimizin bu işlemi başarılı bir şekilde yapacağına inanıyoruz. Umarım sonuçlar bizi yanıltmaz.'
 'Çünkü bu sonuçlar bizim için çok önemli.']
[]


In [407]:
for i in range(len(predicted_results)):
    # Check if length of the predicted sentence is different than the original sentence
    if len(predicted_results[i]) != len(test_sentences[i]):
        print("Original:", test_sentences[i])
        print("Predicted:", predicted_results[i])
        print()

In [408]:
# Save the predictions to a CSV file, it will have two columns: "ID" and "Sentence"
output_df = pd.DataFrame({"ID": nlp_test.index, "Sentence": predicted_results})

ValueError: All arrays must be of the same length

In [ ]:
# Save the dataframe to a CSV file
output_df.to_csv("predictions.csv", index=False)

In [ ]:
output_df

,ID,Sentence
0,0,tr ekonomi ve politika haberleri türkiye nin ...
1,1,üye girişi
2,2,son güncelleme 12:12
3,3,Imralı Mit görüşmesi ihtiyaç duyuldukça oluyor
4,4,Suriye deki silahlı selefi muhalifler yeni ku...
...,...,...
1152,1152,Yüreğir Adana ilimize ait şırın bir ilçedir
1153,1153,yüze gülüçülüğün at oynattığı bir aydınlar ort...
1154,1154,zavallı adamı oracıkta astılar ve hiç kimse se...
1155,1155,zengin çocuklarına arız münasebetsizlikler fak...
